# SPARQL Langchain with Amazon Neptune

This notebook shows use of LLM to query RDF graph in Amazon Neptune. It uses Langchain.

Approach adapted from https://python.langchain.com/docs/use_cases/more/graph/graph_sparql_qa.

* Create RDFGraph object that connects to Neptune and introspects its schema
* Create chain using LLM and RDFGraph. LLM used here is Anthropic Claude 2 via Bedrock
* Ask questions to LLM. LLM consults schema and calls RDFGraph to execute SPARQL query

## Setup

### Notebook Pre-Req

You can upload this notebook into a Jupyter environment configured to use Neptune Workbench. I tested this on an Amazon Sagemaker notebook running Python 3.10.x. 

### Python Pre-Req

I tested this on an Amazon Sagemaker notebook running Python 3.10.x. You need Python 3.9 or higher.

### Neptune Pre-Req

Your Neptune cluster must run engine version 1.2.x or higher

### Bedrock

In Bedrock console, allow access to Anthropic Claude v2 model in this region.
In Notebook IAM role, add

{
        "Action": [
            "bedrock:ListFoundationModels",
            "bedrock:InvokeModel"
        ],
        "Resource": "*",
        "Effect": "Allow"
}

## Seed W3C organizational data
W3C org ontology plus some instances. 

You will need an S3 bucket in the same region and account. Set STAGE_BUCKET to name of that bucket.

In [ ]:
STAGE_BUCKET='haveym-langchain-sparql'

In [ ]:
%%bash  -s "$STAGE_BUCKET"

rm -rf data
mkdir -p data
cd data
echo getting org ontology and sample org instances
wget http://www.w3.org/ns/org.ttl 
wget https://raw.githubusercontent.com/aws-samples/amazon-neptune-ontology-example-blog/main/data/example_org.ttl 

echo Copying org ttl to S3
aws s3 cp org.ttl s3://$1/org.ttl
aws s3 cp example_org.ttl s3://$1/example_org.ttl


Bulk-load the org ttl - both ontology and instances

In [ ]:
%load -s s3://{STAGE_BUCKET} -f turtle --store-to loadres --run

In [ ]:
%load_status {loadres['payload']['loadId']} --errors --details

## Install Langchain
Need python 3.9 or greater plus langchain 0.0.341 ish

In [ ]:
%%bash 
# 3.9 or higher?
python --version

In [ ]:
!pip install langchain

In [ ]:
!pip show langchain

## Patch Langhchain


In [ ]:
%%bash 

rm -rf langchain
mkdir langchain
cd langchain
mkdir graphs
cd graphs
wget https://raw.githubusercontent.com/mhavey/langchain/langchain/libs/langchain/langchain/graphs/neptune_rdf_graph.py
wget https://raw.githubusercontent.com/mhavey/langchain/langchain/libs/langchain/langchain/graphs/__init__.py
cd ..
mkdir chains
mkdir chains/graph_qa
cd chains
wget https://raw.githubusercontent.com/mhavey/langchain/langchain/libs/langchain/langchain/chains/__init__.py
cd graph_qa
wget https://raw.githubusercontent.com/mhavey/langchain/langchain/libs/langchain/langchain/chains/graph_qa/neptune_sparql.py


In [ ]:
%%bash

LCDIR=~/anaconda3/envs/JupyterSystemEnv/lib/python3.10/site-packages/langchain

cp -r langchain/* $LCDIR


### Now restart Kernel *** 

## Setup Chain

In [ ]:
import os

# Grab Neptune cluster host/port from notebook instance environment variables
GRAPH_NOTEBOOK_HOST= os.popen("source ~/.bashrc ; echo $GRAPH_NOTEBOOK_HOST").read().split("\n")[0]
GRAPH_NOTEBOOK_PORT= os.popen("source ~/.bashrc ; echo $GRAPH_NOTEBOOK_PORT").read().split("\n")[0]
[GRAPH_NOTEBOOK_HOST, GRAPH_NOTEBOOK_PORT]

In [ ]:
EXAMPLES="""

<question>
Find organizations.
</question>

<sparql>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX org: <http://www.w3.org/ns/org#> 

select ?org ?orgName where {{
    ?org rdfs:label ?orgName .
}} 
</sparql>

<question>
Find sites of an organization
</question>

<sparql>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX org: <http://www.w3.org/ns/org#> 

select ?org ?orgName ?siteName where {{
    ?org rdfs:label ?orgName .
    ?org org:hasSite/rdfs:label ?siteName . 
}} 
</sparql>

<question>
Find suborganizations of an organization
</question>

<sparql>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX org: <http://www.w3.org/ns/org#> 

select ?org ?orgName ?subName where {{
    ?org rdfs:label ?orgName .
    ?org org:hasSubOrganization/rdfs:label ?subName  .
}} 
</sparql>

<question>
Find organizational units of an organization
</question>

<sparql>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX org: <http://www.w3.org/ns/org#> 

select ?org ?orgName ?unitName where {{
    ?org rdfs:label ?orgName .
    ?org org:hasUnit/rdfs:label ?unitName . 
}} 
</sparql>

<question>
Find members of an organization. Also find their manager, or the member they report to.
</question>

<sparql>
PREFIX org: <http://www.w3.org/ns/org#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 

select * where {{
    ?person rdf:type foaf:Person .
    ?person  org:memberOf ?org .
    OPTIONAL {{ ?person foaf:firstName ?firstName . }}
    OPTIONAL {{ ?person foaf:family_name ?lastName . }}
    OPTIONAL {{ ?person  org:reportsTo ??manager }} .
}}
</sparql>


<question>
Find change events, such as mergers and acquisitions, of an organization
</question>

<sparql>
PREFIX org: <http://www.w3.org/ns/org#> 

select ?event ?prop ?obj where {{
    ?org rdfs:label ?orgName .
    ?event rdf:type org:ChangeEvent .
    ?event org:originalOrganization ?origOrg .
    ?event org:resultingOrganization ?resultingOrg .
}}
</sparql>

"""

In [ ]:
import boto3
from langchain.chains import NeptuneSparqlQAChain
from langchain.graphs import NeptuneRdfGraph
from langchain.chains import ConversationChain
from langchain.chat_models import BedrockChat
from langchain.llms import Bedrock

nep_ep = f"https://{GRAPH_NOTEBOOK_HOST}:{GRAPH_NOTEBOOK_PORT}/sparql"

graph = None
graph = NeptuneRdfGraph(
    query_endpoint=nep_ep,
    use_iam_auth=True,
    region_name='us-east-1',
    hide_comments=True
)

bedrock_client = boto3.client('bedrock-runtime')
llm = BedrockChat(
    model_id = "anthropic.claude-v2",
    client = bedrock_client
)

chain = NeptuneSparqlQAChain.from_llm(
    llm=llm, graph=graph, examples=EXAMPLES, verbose=True, top_K=10, return_intermediate_steps=True, return_direct=False)


In [ ]:
graph.get_schema

In [ ]:
graph.get_schema_elements

In [ ]:
chain

## Ask questions
Depends on the data we ingested above

In [ ]:
chain.run('''How many organizations are in the graph''')

In [ ]:
chain.run('''Are there any mergers or acquisitions''')

In [ ]:
chain.run('''Find organizations''')

In [ ]:
chain.run('''Find sites of MegaSystems or MegaFinancial''')

In [ ]:
%%time
chain.run('''Find a member who is manager of one or more members.''')

In [ ]:
chain.run('''Find five members and who their manager is.''')

In [ ]:
%%time
chain.run('''Find org units or suborganizations of The Mega Group. What are the sites of those units?''')